<a href="https://colab.research.google.com/github/shimossa/ISYS5002_Assignment_One/blob/main/Sandbox_Practice_Assignment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Assignment Ideas**

1. State the problem clearly
2. Describe the input and output information
3. Work the problem by hand
4. Develop and algorithm 
5. Test slution with a variety of data

Program, Inputs, Processing and Outputs

# Inputs

*Input is a ticker symbol?
Note: To prompt the user, I guess I will need to use the input function 

**Find out what a is a ticker symbol is** 
A stock symbol is a unique series of letters assigned to a security for trading purposes. Stocks listed on the New York Stock Exchange (NYSE) can have four or fewer letters.
https://www.investopedia.com/terms/s/stocksymbol.asp


# Processing thoughts 

*scrape the yahoo website
*extract the 'key executives'
*Calculate the income tax

**Find yahoo webstie**
**Find the key executives**

Here is the webpage in which data is to be retrieved 
https://au.finance.yahoo.com/

Note: There is a search bar, will use to find a company

#Outputs

*Save to a CSV file
*Save to an SQL database

**Workout what I have to save**


#Explore the website

1. Search for Coles 

Best guess - COL.AX is Coles 
'.AX' means the company is Australian

Here is the URL for Coles
https://au.finance.yahoo.com/quote/COL.AX/profile?p=COL.AX

2. Find NAB

Here is the URL for NAB
https://au.finance.yahoo.com/quote/NAB.AX/profile?p=NAB.AX

There appears to be a URL pattern - the ticker is used twice in the URL.


https://au.finance.yahoo.com/quote/____/profile?p=____


Manual process is
1. Search for the ticker
2. Click on 'Profile'

**TODO: Can this process be shortcutted and just use the URL**

The URL can be edited to replace the ticker - only the first mention of the ticker is important, but to be safe both should be replaced.

#Lets manually calculate the income tax

```

Name	Title	Pay	Exercised	Year born
Mr. Ross Maxwell McEwan CBE, B.B.S.	Group CEO, MD & Director	2.54M	N/A	1957
Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons), FCA	Group Chief Financial Officer	1.13M	N/A	N/A
Mr. Patrick F. Wright	Group Exec. of Technology & Enterprise Operations	1.72M	N/A	1967
Ms. Sharon J. Cook	Group Exec. of Legal & Commercial Services	912.98k	N/A	N/A
Ms. Susan Ferrier	Group Exec. of People & Culture	929.27k	N/A	N/A
Ms. Angela Mentis B.Bus	Group Chief Digital, Data & Analytics Officer	1.62M	N/A	N/A
Mr. Shaun Dooley	Chief Risk Officer	1.05M	N/A	N/A
Mr. Andrew Irvine	Chief Exec. of Bus. Private Banking Division	333.27k	N/A	N/A
Mr. Lesile D. Matheson	Group Chief Operating Officer	N/A	N/A	1960
Ms. Sally Mihell	Head of Investor Relations	N/A	N/A	N/A

```

The first column is name
The second is Their title in the organisation
The third is their pay

**TODO: How to do we extract the third column***


Only need the name, title and pay
```
Mr. Ross Maxwell McEwan CBE, B.B.S.	Group CEO, MD & Director	2.54M	N/A	1957
```

Thrid column is pay; 2.54m

M is assumedly "Million"; The pay being 2.5 * 1000000



In [1]:
2.5*1000000

2500000.0

Inspect another row for feasibility 
```
Ms. Sharon J. Cook	Group Exec. of Legal & Commercial Services	912.98k	N/A	N/A
```

Noticed the pay is 912.8k
K assumedly meaning thousand or 1000; therefore their pay is 912.8*1000

In [3]:
912.8*1000

912800.0

Checking to see if any other rows are different 

```
Mr. Lesile D. Matheson	Group Chief Operating Officer	N/A	N/A	1960
```

What do I do about N/A

```
def calculate_tex(income):
  if income <= 18200:
    tax_payable = 0
  elif 18200 < income <= 45000:
    tax_payable = 0.19*(income - 18200)
  elif 45000 < income < 120000:
    tax_payabnle =  5092 + 0.32 * (income - 45000)

  return tax_payable
```

#Initial Algorithm 

1. User input ticker symbol
2. Use ticker to get profile page 
3. Copied the table 
4. For each row of the table 
  - Process a row to extract pay
  - Find the last character to determine multiplier 
  - Calculate salary, pay multiplier 
  - Calculate the income tax

NOTE: 
  1. Need to validate the ticker but for now  Iwill assume the user type in a correct ricker 
  2. Still need to work out where I save the imcome tax to 

In [4]:
#User input ticker symbol 
ticker_symbol = input("Please input ticker symbol ")
print(ticker_symbol)

Please input ticker symbol ANB.AX
ANB.AX


In [8]:
#Use ticker to get profile page

example_url = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
example_url.format(ticker_symbol, ticker_symbol)

'https://au.finance.yahoo.com/quote/ANB.AX/profile?p=ANB.AX'

In [9]:
def build_url():
  #user inout ticker symbol
  ticker_symbol = input("Please input ticker symbol: ")
  print(ticker_symbol)
  example_url = 'https:////au.finance.yahoo.com/quote/{}/profile?p={}'
  result = example_url.format(ticker_symbol, ticker_symbol)
  return result

In [12]:
url = build_url()
print(url)

Please input ticker symbol: COL.AX
COL.AX
https:////au.finance.yahoo.com/quote/COL.AX/profile?p=COL.AX
